In [8]:
import csv
import numpy as np
from pathlib import Path
import pandas as pd
from pathlib import Path
import pandas as pd
import numpy as np
import subprocess, json, tqdm, os
import torch, torchaudio, torchvision.transforms as T
import torchaudio, transformers, torch
import random
from typing   import List, Tuple
# import cv2

from torchcodec.decoders import VideoDecoder              # torchcodec 0.3.0
from transformers        import AutoModel, AutoProcessor   # transformers 4.x

If you have access to the cloned github/video data. Run this as a sanity check that you have access to the vids

In [9]:
# file = "image-data/9s4baJE29mP6-0Nk-KJRoS7TV34.jpg"
def read_csv(path):
    with open (path) as rf:
        reader = csv.DictReader(rf)
        rows = []
        # print(reader)
        row_lens = dict()
        for row in reader:
            rows.append(row)
            # print(row)
            if not ((len(row)) in row_lens.keys()):
                row_lens[len(row)] = 1
            else:
                row_lens[len(row)] += 1
        return rows
    csv.DictReader() 
    
def check_all_names_found (media_file, metadata_file):
    imgFileList = os.listdir(media_file)
    csv_rows = read_csv(metadata_file)[1::]
    found_count = 0
    not_found_count = 0
    found_files = []
    not_found = []
    for r in csv_rows:
        # print(r)
        name = r["Filename"]
        if (name in imgFileList):
            found_count += 1
            found_files.append(name)
            # if ()
        else:
            not_found_count += 1
            not_found.append(name)
        # print(r["Filename"])
    # print(csv_rows)
    print(f"FOUND: {found_count}")
    print(f"NOT FOUND #: {not_found_count}")
    print(f"NOT FOUND #: {not_found}")
    return np.array(found_files)

def load_vid(file):
    file = "video-data/" + file
    cap = cv2.VideoCapture(file)
    #get all of the dimensions of the video for the numpy array
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    rgb_count = np.zeros((frame_count, height, width, 3), dtype = np.uint8)
    
    if not cap.isOpened():
        raise IOError("Cannot open vid")
    for i in tqdm(range(frame_count)):
        ret, frame = cap.read()
    
        # print(f"Frame length of vid: {frame_count}")
        if not ret:
            cap.release()
            return
        
        rgb_count[i] = frame[..., ::-1] ## cv2 tyrants used bgr instead of rgb, need to invert it 
    
    height, width = frame.shape[:2]
    print(f"height: {height}")
    print(f"width: {width}")
    return rgb_count


def load_vid_list(files, label):
    num = len(files)
    label_col = np.full(num, label)
    
    for i in tqdm(range(num)):
        cap = cv2.VideoCapture("video-data/" + files[i])
        if not cap.isOpened():
            raise IOError("Cannot open vid")
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        
        rgb_count = np.zeros((frame_count, height, width, 3), dtype = np.uint8)
        
        for j in range(frame_count):
            ret, frame = cap.read()
            if not ret:
                print(f"video that failed: video-data/{files[i]}, skipping now")
                i += 1
                break
            rgb_count[j] = frame[..., ::-1]
        
    
    return rgb_count




In [10]:
clean_vids = check_all_names_found("video-data/", "video-metadata-publish.csv")

def clear_unknowns(clean_vids, metadata_file):
    csv_rows = read_csv(metadata_file)
    bof = {"Real": [], "Fake": []}
    for row in csv_rows:
        vL = row["Video Ground Truth"]
        aL = row["Audio Ground Truth"]
        if vL == "Real" and aL == "Real":
            bof["Real"].append(row["Filename"])
        elif vL == "Fake" and aL == "Fake":
            bof["Fake"].append(row["Filename"])
    return bof

FOUND: 2035
NOT FOUND #: 0
NOT FOUND #: []


In [11]:
clean_vids = check_all_names_found("video-data/", "video-metadata-publish.csv")
bof = clear_unknowns(clean_vids, "video-metadata-publish.csv")
print(bof)
print(f"Len of real_real: {len(bof["Real"])}")
print(f"Len of fake_fake: {len(bof["Fake"])}")

FOUND: 2035
NOT FOUND #: 0
NOT FOUND #: []
{'Real': ['HICemuEclIn-ZDbqZVUULNCfNMI.mp4', 'kAaNHcuBnFPnxe4tspbxc_muFTw.mp4', '7QkcRIbGeqPx-3lRLNVEoF3Z9MM.mp4', 'rN7AySTzGxCHYrDYJcrqqN9ybNc.mp4', 'YBy7OnWBQ3GOQzm53dEEQv3_YBk.mp4', 'nefJI_4TbHarNctRfXP_pXxf4fg.mp4', '5lFghVsgAMh2mS_6lW4dNxa3Ma0.mp4', '_jB8v1mKu-G-YtCcsbCDz-Xqy7I.mp4', 'ZhGDGjAEcmCeKMaL7n2EHc4QtO8.mp4', 'C2itTABNRmzZt3NMWX5xtgV2E-g.mp4', 'JcazzdNzkDMktnDR6iORAoJzRNs.mp4', 'KNRJMAZi9TCdajnupiEMw4vDyl0.mp4', 'tMZRLQlh4tgT49AIODi_H44tHTA.mp4', 'S-M6eZeqaT8FOPEgHpUeboW4DIc.mp4', 'DtE_ZtU_z3fU4DSIvU4h9COtvno.mp4', 'lwf0nbH97hWa6zRYnkyRtIvynzQ.mp4', '5_sVFeQ4JyHtu2bIysyApXAtDyI.mp4', 'qtusd7VLnWcHTJtQjmVjTednP3c.mp4', '_-v4jJibKTSv-slQIrt8DfnwrRU.mp4', 'ylUxctRpgu_Aqya7Mg268MLaf5M.mp4', 'CV8UPfKVejNrpp5mgEcJQibabCw.mp4', '-wHhS6r-xuXnfOId95KMz5YTuJ4.mp4', 'fnGAuF4EzmmT1UhGMMHv2MwrOW8.mp4', 'hGah7PGiNDWpPJFWDGizAas1hzU.mp4', 'jBj3uM9aJU_N2Q6Lafb3BJS_S6g.mp4', 'TVPSrhfYU9_VIS47bAehJggz83c.mp4', 'kiuaxEn9qpF8h4WbN064opCsPN4.mp4', '1

Make a manifest csv to share what videos we will use for the dataset.


In [12]:

base_dir   = Path("video-data/")
real_files = bof["Real"]               
fake_files = bof["Fake"]

def make_manifest(names: List, label: int):
    full_paths: List = []
    for file in names:
        full_path = base_dir / file  
        full_paths.append(full_path.as_posix())
    df = pd.DataFrame({
        "path":  full_paths,
        "label": label
    })
    return df

manifest = pd.concat([make_manifest(real_files, 0), make_manifest(fake_files, 1)], ignore_index=True)

def video_duration_tc(path):
    """
    Returns clip duration by seconds (roughly).
    """
    vd  = VideoDecoder(path)
    meta = vd.metadata 
    # print(meta)

    if (not (meta.average_fps is None)) and meta.average_fps > 0:
        return len(vd) / meta.average_fps

    # 2) header already stores duration? (not always present)
    if meta.duration is not None:
        return meta.duration

    # 3) as a last resort, decode one packet to query its PTS
    raise RuntimeError("no duration information in header")

# ---------------------------------------------------------------------
# 3.  Enrich manifest with duration & drop unreadable clips
# ---------------------------------------------------------------------
durations, bad_indices = [], []

# any time you see tqdm its just to make the progress bar, dont worry about it
for idx, row in tqdm.tqdm(manifest.iterrows(), total=len(manifest)):
    try:
        durations.append(video_duration_tc(row["path"]))
    except Exception as e:
        durations.append(np.nan)
        bad_indices.append(idx)
        print(f"BAD VIDEO {row['path']} unreadable erorr is: {e}")
    # print("adding")
manifest["duration_s"] = durations

if bad_indices:
    manifest.drop(index=bad_indices, inplace=True)
    print(f"dropped {len(bad_indices)} because they couldnt be opend")

manifest.to_csv("manifest.csv", index=False)

#-------Some videos are way too long, need to remove outliers-----------------------------------------------------
df = pd.read_csv("manifest.csv")   # or "manifest.csv"

dur = df["duration_s"].to_numpy()

print(f"MEAN: {dur.mean():.2f} s")
print(f"MEDIAN: {np.median(dur):.2f} s")
print(f"Q1: {np.quantile(dur, .25):.2f} s")
print(f"Q3: {np.quantile(dur, .75):.2f} s")
Q50 = np.quantile(dur, .50) # median lol 
Q75 = np.quantile(dur, .75)
Q95 = np.quantile(dur, .95)
Q25 = np.quantile(dur, .25)
print(f"Q.95: {Q95:.2f} s")
print(f"MAX vid length : {dur.max():.2f} s")


before = len(df)
# df = df[df["duration_s"] <= Q95].reset_index(drop=True) If I wanted to 
df = df[df["duration_s"] <= Q75].reset_index(drop=True) # drop the ones that have about Q threshold (pruning super long vids)
after  = len(df)

df.to_csv("manifest_pruned_outliers.csv", index=False)
print("Pruned videos that are outliers/too long")
print(f"({after}/{before} remain)")
print("Filtered manifest ➜ manifest_pruned_outliers.csv")

100%|██████████| 424/424 [00:05<00:00, 81.55it/s] 


MEAN: 90.70 s
MEDIAN: 43.37 s
Q1: 21.64 s
Q3: 75.23 s
Q.95: 250.03 s
MAX vid length : 5632.87 s
Pruned videos that are outliers/too long
(318/424 remain)
Filtered manifest ➜ manifest_pruned_outliers.csv


Make the models

In [ ]:
# read rgb vals for a video and store them in a tensor
_img_tx = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ConvertImageDtype(torch.float32),  # → [0,1]
    T.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5]) # → recenetering [-1, 1]
])

def read_frames(path, fps: int = 2):
    """
    Decode `path` with torchcodec.VideoDecoder and return a tensor of
    shape (T, 3, 224, 224) normalised to [-1,1].
    """
    vr = VideoDecoder(path, dimension_order="NHWC") # each is (H,W,C)

    src_fps = (
        vr.metadata.average_fps
        or vr.metadata.average_fps_from_header
        or 30.0   # fall back in case nothing else is true
    )
    step = max(int(round(src_fps / fps)), 1) ## make the parameter fps super low if you want to increase video speed, lose data from this though

    frames = []
    for f in vr[::step]: # move through video in our step size
        if isinstance(f, np.ndarray):
            f = torch.from_numpy(f) #converts numpys to tensor
        frames.append(_img_tx(f.permute(2, 0, 1))) # reorder  to C,H,W
    return torch.stack(frames)       


#Audio section---------------------------

# helper to use torchaudio if it works for the vid, if not use ffmpeg
def _load_audio_any(path, sr: int = 16_000, mono: bool = True):
    try:
        wav, sr0 = torchaudio.load(path)          # tries 'soundfile' backend
        if sr0 != sr:
            wav = torchaudio.functional.resample(wav, sr0, sr)
        if mono and wav.shape[0] > 1:
            wav = wav.mean(0, keepdim=True)
        return wav, sr
    except Exception:
        # ffmpeg fallback 
        cmd = [
            "ffmpeg", "-i", str(path),
            "-f", "f32le", "-ac", "1" if mono else "2",
            "-ar", str(sr), "-"
        ]
        pcm = subprocess.check_output(cmd, stderr=subprocess.DEVNULL)
        wav = torch.frombuffer(pcm, dtype=torch.float32)
        wav = wav.view(-1) if mono else wav.view(-1, 2).t()
        return wav.unsqueeze(0), sr                # (1,N), sr

# 3.2 wav2vec2 model & processor (loaded once)
wav2vec = AutoModel.from_pretrained("facebook/wav2vec2-base").eval() # pre-trained voice recognition model. Seems to be a standard
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

@torch.inference_mode()
def read_audio_embed(path, target_T, sr: int = 16_000) -> torch.Tensor:
    """
    Return a (T,768) feature sequence aligned to the number of video frames.
    If audio decoding fails, a zero-tensor of the right shape is returned.
    """
    try:
        wav, _ = _load_audio_any(path, sr=sr, mono=True)   # (1,N)
        feats  = wav2vec(**processor(wav.squeeze(), sampling_rate=sr, return_tensors="pt")).last_hidden_state.squeeze(0)            # (N',768)
    except Exception as e:
        print("audio failed, putting 0's in its place") # This shoudlnt happen anymore now that I filtered them out
        feats = torch.zeros(target_T, 768)
        return feats

    # temporal alignment to video: linear index sampling
    idx = torch.linspace(0, feats.size(0)-1, target_T).round().long()
    return feats[idx]  # (T,768)


# 50% cross modal masks like the report says we'll do
def make_masks(T, p = .5):
    idx = torch.randperm(T)
    n = int(T * p)
    vmask = torch.zeros(T, dtype=torch.bool)
    amask = torch.zeros(T, dtype=torch.bool)
    vmask[idx[: n//2]] = True
    amask[idx[n//2: n]] = True
    return vmask, amask #(torch.Tensor, torch.Tensor)


#Dataset wrapper
class AVDeepfakeDataset(torch.utils.data.Dataset):
    def __init__(self, csv, fps: int = 2): #default fps if none is passed in
        self.df  = pd.read_csv(csv)
        self.fps = fps
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        path  = self.df.at[idx, "path"]
        label = self.df.at[idx, "label"]
        frames = read_frames(path, fps=self.fps)
        audio  = read_audio_embed(path, frames.size(0))
        vmask, amask = make_masks(frames.size(0))
        return frames, audio, vmask, amask, label


/opt/anaconda3/envs/avdf/lib/python3.12/site-packages/transformers/configuration_utils.py:309: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


I ran smth to find these and print them but it took 30+ mins to run on like 200 vids so I'm leaving it out of this code

In [14]:
audios_dont_load = ["video-data/fZ09ZAlhUdYHAu60Cufv1x2y20M.mp4", "video-data/sNJIBQDVLVDPmp1J28pmG5dmuD8.mp4","video-data/F3dz0Dfm3Se6NN9QbL9h-HaHZSY.mp4",
"video-data/uUIKWQ9REZjhJgPagpiAbpUfqkI.mp4","video-data/YfX9oF_Fka8ITOhlO0Kh8rz9T60.mp4","video-data/oesaw0deyLmtIflYANhgy8RL0qY.mp4",
"video-data/KewDIqJQ-jubya1H-mOUBn1SMAg.mp4", "video-data/JXKMuwxTdJtnprXysiSjDiLlywE.mp4"]

# prunedf = pd.read_csv("manifest_pruned_outliers.csv")
# # print(prunedf)
# print(prunedf["path"])

prunedf = pd.read_csv("manifest_pruned_outliers.csv")
print(len(audios_dont_load))
print(len(prunedf))

# keep every row **whose path is NOT in** audios_dont_load
prunedf = prunedf[~prunedf["path"].isin(audios_dont_load)]

print("Rows left :", len(prunedf))
prunedf.to_csv("final_filtered_manifest.csv", index=False)

8
318
Rows left : 310


Encoding video features with resnet50 pretrained and store them so you all are able to read the features

In [18]:
from pathlib import Path
import torch, tqdm
from torchvision.models import resnet50, ResNet50_Weights

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using", device)

cnn = resnet50(weights=ResNet50_Weights.DEFAULT).to(device).eval()
feat_extractor = torch.nn.Sequential(*(list(cnn.children())[:-1]))

def rgb2feat(frames):
    with torch.inference_mode():
        f = feat_extractor(frames.to(device))
        return f.squeeze(-1).squeeze(-1).cpu()


ds = AVDeepfakeDataset("stored_features.csv", fps=2) #higher fps means more data, you can edit this
out = Path("feature_cache")
out.mkdir(exist_ok=True)

for i in tqdm.tqdm(range(len(ds)), desc="export"):
    frames, audio, vmask, amask, y = ds[i]

    key = Path(ds.df.at[i, "path"]).stem
    v_feat = rgb2feat(frames)  # (T,2048)
    a_feat = audio  # already (T,768)

    torch.save(
        {
            "vision": v_feat.half(),      # float16 to halve the size
            "audio" : a_feat.half(),
            "vmask" : vmask,
            "amask" : amask,
            "label" : y
        }, out / f"{key}.pt" # stored in feature_cache folder
    )

Using cpu


export: 100%|██████████| 78/78 [04:54<00:00,  3.78s/it]


In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch

class FeatureDataset(torch.utils.data.Dataset):
    def __init__(self, feat_dir):
        self.paths = sorted(Path(feat_dir).glob("*.pt"))
    def __len__(self): 
        return len(self.paths)
    def __getitem__(self, idx):
        pkg = torch.load(self.paths[idx], map_location="cpu", weights_only=False)
        
        return pkg["vision"], pkg["audio"], pkg["vmask"], pkg["amask"], int(pkg["label"])

loader = torch.utils.data.DataLoader(FeatureDataset("feature_cache"),
                            batch_size=32, shuffle=True)

def collate_pad(batch):
    v_list, a_list, vm_list, am_list, y_list = zip(*batch)

    # --- pad features -------------------------------------------------
    v_pad = pad_sequence(v_list, batch_first=True)
    a_pad = pad_sequence(a_list, batch_first=True)

    # --- create padding mask (True where we inserted padding) ---------
    lengths = torch.tensor([x.size(0) for x in v_list])
    T_max   = v_pad.size(1)
    pad_msk = torch.arange(T_max)[None, :] >= lengths[:, None]   # (B,T_max)

    # pad+OR with existing modality masks
    vm_pad = pad_sequence(vm_list, batch_first=True) | pad_msk
    am_pad = pad_sequence(am_list, batch_first=True) | pad_msk

    labels = torch.tensor(y_list)

    return v_pad, a_pad, vm_pad, am_pad, labels


# ------------------------------------------------------------------
# Loader that can be used wiht the model (given that you have the "feature_cache" folder downlaoded and in the project)
ds = FeatureDataset("feature_cache")
loader = DataLoader(
    ds,
    batch_size   = 4,
    shuffle      = True,
    num_workers  = 0,
    collate_fn   = collate_pad,
    pin_memory   = False  # no CUDA 
)

#load a batch's dims for testing 
v_b, a_b, vm_b, am_b, y_b = next(iter(loader))
print("vision :", v_b.shape)   # (B, T_max, 2048)
print("audio  :", a_b.shape)   # (B, T_max, 768)
print("video_mask  :", vm_b.shape)
print("audio_mask :", am_b.shape)
print("labels :", y_b)

vision : torch.Size([4, 27, 2048])
audio  : torch.Size([4, 27, 768])
video_mask  : torch.Size([4, 27])
audio_mask : torch.Size([4, 27])
labels : tensor([1, 1, 1, 1])


In [22]:
from pathlib import Path
import torch, tqdm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# -- 0.  dataset & collate ----------------------------------------------------
class FeatureDataset(torch.utils.data.Dataset):
    def __init__(self, feat_dir):
        self.paths = sorted(Path(feat_dir).glob("*.pt"))
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        pkg = torch.load(self.paths[idx], map_location="cpu", weights_only=False)
        return (pkg["vision"], pkg["audio"],
                pkg["vmask"], pkg["amask"], int(pkg["label"]))

def collate_pad(batch):
    v_list, a_list, vm_list, am_list, y_list = zip(*batch)

    v_pad = pad_sequence(v_list, batch_first=True)      # (B, T_max, 2048)
    a_pad = pad_sequence(a_list, batch_first=True)      # (B, T_max, 768)

    lens      = torch.tensor([x.size(0) for x in v_list])
    T_max     = v_pad.size(1)
    pad_mask  = torch.arange(T_max)[None] >= lens[:, None]   # True where padded

    vm_pad = pad_sequence(vm_list, batch_first=True) | pad_mask
    am_pad = pad_sequence(am_list, batch_first=True) | pad_mask
    labels = torch.tensor(y_list)

    return v_pad, a_pad, vm_pad, am_pad, labels

# -- 1.  loader ---------------------------------------------------------------
feat_root = "feature_cache"
loader = DataLoader(FeatureDataset(feat_root),
                    batch_size   = 32,
                    shuffle      = False,   # deterministic pass
                    num_workers  = 0,
                    pin_memory   = False,
                    collate_fn   = collate_pad)

# -- 2.  scan everything ------------------------------------------------------
n_batches = 0
n_samples = 0
n_frames  = 0    # total (unpadded) time-steps for info only

with tqdm.tqdm(loader, desc="verifying") as bar:
    for v, a, vm, am, y in bar:
        # ---- quick consistency checks every batch --------------------------
        B, T, _ = v.shape
        assert a.shape == (B, T, 768)
        assert vm.shape == am.shape == (B, T)
        # assert (vm & am).sum() == 0, "positions cannot be masked twice"
        assert torch.isfinite(v).all() and torch.isfinite(a).all()

        # statistics
        n_batches += 1
        n_samples += B
        n_frames  += (vm == False).sum().item()   # “kept” frames (≈ real length)

        bar.set_postfix(batch=n_batches, samples=n_samples)

print(f"\n✅  finished without errors:"
    f"\n    • batches  : {n_batches}"
    f"\n    • samples  : {n_samples}"
    f"\n    • frames   : {n_frames:,}")


verifying: 100%|██████████| 3/3 [00:00<00:00, 41.40it/s, batch=3, samples=80]


✅  finished without errors:
    • batches  : 3
    • samples  : 80
    • frames   : 1,435
